In [1]:
!pip install fuzzywuzzy
!pip install transformers
!pip install PyPDF2
!pip install textract
import json 

# data manipulation
import pandas as pd 
import string
# normalize nested JSON files
from pandas.io.json import json_normalize
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
#import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import json
import logging
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import re
import regex
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')  
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 4.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 118.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
JSON_FEATURE_DIR = "/content/drive/MyDrive/RAjobs/try"


In [ ]:
import ast

In [4]:
def content(jsonfile):
    curr_id = jsonfile.split('.')[0]
    with open(os.path.join(JSON_FEATURE_DIR, jsonfile)) as f:
        raw_json = json.load(f)

        ## for new data
        raw_json = eval("["+raw_json['text']+"]")  ## convert str to dict
        raw_json = [{'section_title': x['header'], 'text': x['text']} for x in raw_json]  ## clean data
        ## for new data
        
        raw_text = list(map(lambda x: ' '.join(x.values()), raw_json))
        raw_text = ' '.join(raw_text)
        raw_text = ' '.join(raw_text.split())
    sentences = '\n'.join(sent_tokenize(raw_text))
    return curr_id, sentences

In [5]:
with mp.Pool(4) as pooler:
    content_dicts = list(pooler.map(content, os.listdir(JSON_FEATURE_DIR)))

In [6]:
content_df = pd.DataFrame(content_dicts)
content_df = content_df.rename(columns={content_df.columns[0]: "ID",content_df.columns[1]: "content"})
content_list = content_df['content'].values.tolist()


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
from nltk import SnowballStemmer
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stemmer = SnowballStemmer("english")
def tokenize(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))  
    return [stemmer.stem(i) for i in text.translate(translator).split()]  

In [9]:
stop_words = set(stopwords.words('english'))
stop_words = [tokenize(s)[0] for s in stop_words]

In [10]:
vectorizer = CountVectorizer(analyzer="word",        
                            tokenizer=tokenize,      
                            ngram_range=(0,1),       
                            strip_accents='unicode', 
                            stop_words = stop_words,
                            min_df = 0.05,           
                            max_df = 0.95)           

In [ ]:

bag_of_words = vectorizer.fit_transform(content_list) 
features = vectorizer.get_feature_names_out()               


transformer = TfidfTransformer(norm = None, smooth_idf = True, sublinear_tf = True)
tfidf = transformer.fit_transform(bag_of_words)

lda = LatentDirichletAllocation(n_components = 5, learning_method='online')
doctopic = lda.fit_transform(bag_of_words)


ls_keywords = []
for i,topic in enumerate(lda.components_):
    word_idx = np.argsort(topic)[::-1][:10]
    keywords = ', '.join(features[i] for i in word_idx)
    ls_keywords.append(keywords)
    print(i, keywords)

In [ ]:
pd.DataFrame(doctopic, columns = ls_keywords)

In [ ]:
topics_doc = pd.DataFrame(doctopic, columns = ls_keywords)
doc_topics_content_id = pd.concat([topics_doc,content_df],axis=1)
doc_topics_content_id.head()

In [14]:
doc_topics_project_id_with_max = pd.concat([topics_doc.idxmax(axis=1),content_df],axis=1).rename(columns={0:'topic'})

In [15]:
topic_group = doc_topics_project_id_with_max.groupby("topic")


In [ ]:
topic_group.get_group('oregon, manag, use, product, research, anim, studi, feed, forest, fire')

In [ ]:
topic_group.get_group('4, h, youth, project, counti, program, learn, member, scienc, introduct')

In [ ]:
topic_group.get_group('plant, soil, water, use, seed, fruit, dri, 1, grow, crop')

In [ ]:
topic_group.get_group('tree, plant, expert, garden, ask, get, oregon, bee, answer, extens')

In [ ]:
topic_group.get_group('oregon, extens, garden, program, communiti, osu, food, counti, master, state')

In [21]:
doc_topics_project_id_with_max.to_excel('/content/drive/MyDrive/RAjobs/textanalysis_OSU.xlsx' ,index= False)